In [ ]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI
from langchain.callbacks import LangChainTracer
from langsmith import Client
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from typing import List, Union
import clingo
import json
import os

# Set your OpenAI API key

 
llm = ChatGroq(
    model="llama3-groq-8b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
 
# Initialize LangSmith client
client = Client()

# Step 1: Convert natural language to atomic facts
nl_to_facts_prompt = PromptTemplate(
    input_variables=["nl_input"],
    template="Convert the following natural language into atomic facts:\n{nl_input}\n\nAtomic facts:"
)

nl_to_facts_chain = LLMChain(
    llm=llm,
    prompt=nl_to_facts_prompt,
    verbose=True,
    tags=["nl_to_facts"],
)

# Step 2: Add rules to form complete ASP program
asp_rules_prompt = PromptTemplate(
    input_variables=["atomic_facts"],
    template="Given these atomic facts:\n{atomic_facts}\n\nAdd rules to form a complete ASP program:"
)

asp_rules_chain = LLMChain(
    llm=llm,
    prompt=asp_rules_prompt,
    verbose=True,
    tags=["asp_rules"],
)

# Step 3: Debug ASP program
debug_asp_prompt = PromptTemplate(
    input_variables=["asp_program"],
    template="Debug the following ASP program and correct any syntax errors:\n{asp_program}\n\nCorrected ASP program:"
)

debug_asp_chain = LLMChain(
    llm=llm,
    prompt=debug_asp_prompt,
    verbose=True,
    tags=["debug_asp"],
)

# Step 4: Run ASP program
def run_asp_program(asp_program: str) -> List[str]:
    ctl = clingo.Control()
    ctl.add("base", [], asp_program)
    ctl.ground([("base", [])])
    models = []
    with ctl.solve(yield_=True) as handle:
        for model in handle:
            models.append(str(model))
    return models

# Step 5: Choose best answer
choose_answer_prompt = PromptTemplate(
    input_variables=["asp_results", "original_question"],
    template="Given these ASP results:\n{asp_results}\n\nAnd the original question:\n{original_question}\n\nChoose the best answer:"
)

choose_answer_chain = LLMChain(
    llm=llm,
    prompt=choose_answer_prompt,
    verbose=True,
    tags=["choose_answer"],
)

# Define tools for the agent
tools = [
    Tool(
        name="NL to Facts",
        func=nl_to_facts_chain.run,
        description="Convert natural language to atomic facts"
    ),
    Tool(
        name="ASP Rules",
        func=asp_rules_chain.run,
        description="Add rules to form complete ASP program"
    ),
    Tool(
        name="Debug ASP",
        func=debug_asp_chain.run,
        description="Debug the ASP program and correct any syntax errors"
    ),
    Tool(
        name="Run ASP",
        func=run_asp_program,
        description="Run the ASP program and return results"
    ),
    Tool(
        name="Choose Answer",
        func=choose_answer_chain.run,
        description="Choose the best answer from ASP results"
    )
]

# Define agent prompt
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

agent_prompt = CustomPromptTemplate(
    template="""Answer the following question as best you can:

Question: {input}

You have access to the following tools:

{tools}

Use the following format:

Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}""",
    tools=tools,
)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=agent_prompt)

# Define agent
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=None,
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

# Create agent executor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Function to process the JSON file and run the pipeline
def process_json_file(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    results = []
    for item in data:
        context = item.get('context', '')
        question = item.get('question', '')
        predicted_asp = item.get('predicted', '')

        # Run the pipeline
        with LangChainTracer(client=client) as tracer:
            atomic_facts = nl_to_facts_chain.run(nl_input=question)
            asp_program = asp_rules_chain.run(atomic_facts=atomic_facts)
            corrected_asp = debug_asp_chain.run(asp_program=asp_program)
            asp_results = run_asp_program(corrected_asp)
            best_answer = choose_answer_chain.run(asp_results=asp_results, original_question=question)

        results.append({
            "context": context,
            "question": question,
            "predicted": predicted_asp,
            "actual_answer": item.get('answer', ''),
            "atomic_facts": atomic_facts,
            "asp_program": asp_program,
            "corrected_asp": corrected_asp,
            "asp_results": asp_results,
            "best_answer": best_answer
        })

    return results

# Process the JSON file and save the results
input_json_file = "results_optimized.json"
output_json_file = "results_with_pipeline_output.json"
results = process_json_file(input_json_file)

with open(output_json_file, 'w') as file:
    json.dump(results, file, indent=4)

print(f"Processing complete. Results saved to {output_json_file}")

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import Tool
import clingo
from typing import List

# Initialize OpenAI LLM
llm = OpenAI(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",

    model_type='chat',
    max_tokens=4096,
    temperature=0.1,
    top_p=1,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stream=False,
)

# Step 1: Convert natural language to atomic facts
nl_to_facts_prompt = PromptTemplate(
    input_variables=["nl_input"],
    template="Convert the following natural language into atomic facts:\n{nl_input}\n\nAtomic facts:"
)

nl_to_facts_chain = LLMChain(
    llm=llm,
    prompt=nl_to_facts_prompt,
    verbose=True,
)

# Step 2: Add rules to form complete ASP program
asp_rules_prompt = PromptTemplate(
    input_variables=["atomic_facts"],
    template="Given these atomic facts:\n{atomic_facts}\n\nAdd rules to form a complete ASP program:"
)

asp_rules_chain = LLMChain(
    llm=llm,
    prompt=asp_rules_prompt,
    verbose=True,
)

# Step 3: Debug ASP program
debug_asp_prompt = PromptTemplate(
    input_variables=["asp_program"],
    template="Debug the following ASP program and correct any syntax errors:\n{asp_program}\n\nCorrected ASP program:"
)

debug_asp_chain = LLMChain(
    llm=llm,
    prompt=debug_asp_prompt,
    verbose=True,
)

# Step 4: Run ASP program
def run_asp_program(asp_program: str) -> List[str]:
    ctl = clingo.Control()
    ctl.add("base", [], asp_program)
    ctl.ground([("base", [])])
    results = []
    with ctl.solve(yield_=True) as handle:
        for model in handle:
            results.append(str(model))
    return results

# Step 5: Choose best answer
choose_answer_prompt = PromptTemplate(
    input_variables=["asp_results", "original_question"],
    template="Given these ASP results:\n{asp_results}\n\nAnd the original question:\n{original_question}\n\nChoose the best answer:"
)

choose_answer_chain = LLMChain(
    llm=llm,
    prompt=choose_answer_prompt,
    verbose=True,
)

# Main pipeline function
def pipeline(nl_input: str, original_question: str) -> str:
    # Step 1: Convert natural language to atomic facts
    atomic_facts = nl_to_facts_chain.run(nl_input=nl_input)

    # Step 2: Add rules to form complete ASP program
    asp_program = asp_rules_chain.run(atomic_facts=atomic_facts)

    # Step 3: Debug ASP program
    debugged_asp_program = debug_asp_chain.run(asp_program=asp_program)

    # Step 4: Run ASP program
    asp_results = run_asp_program(debugged_asp_program)

    # Step 5: Choose the best answer
    best_answer = choose_answer_chain.run(asp_results=asp_results, original_question=original_question)

    return best_answer

# Example usage
if __name__ == "__main__":
    nl_input = "A person wants to know if they can go to the park."
    original_question = "Can I go to the park if it is raining?"
    
    answer = pipeline(nl_input, original_question)
    print("Best Answer:", answer)

In [ ]:
from langchain import Runnable, RunnablePassthrough, LangChainPredict, StrOutputParser, Memory
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import clingo

# Initialize OpenAI LLM
llm = OpenAI(
    model="deepseek-chat",
    api_base="https://api.deepseek.com",
    api_key="",
    model_type='chat',
    max_tokens=4096,
    temperature=0.1,
    top_p=1,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stream=False,
)

# Step 1: Convert natural language to atomic facts
nl_to_facts_prompt = PromptTemplate(
    input_variables=["nl_input"],
    template="Convert the following natural language into atomic facts:\n{nl_input}\n\nAtomic facts:"
)

# Step 2: Add rules to form complete ASP program
asp_rules_prompt = PromptTemplate(
    input_variables=["atomic_facts"],
    template="Given these atomic facts:\n{atomic_facts}\n\nAdd rules to form a complete ASP program:"
)

# Step 3: Debug ASP program
debug_asp_prompt = PromptTemplate(
    input_variables=["asp_program"],
    template="Debug the following ASP program and correct any syntax errors:\n{asp_program}\n\nCorrected ASP program:"
)

# Step 4: Run ASP program
def run_asp_program(asp_program: str) -> str:
    ctl = clingo.Control()
    ctl.add("base", [], asp_program)
    ctl.ground([("base", [])])
    results = []
    with ctl.solve(yield_=True) as handle:
        for model in handle:
            results.append(str(model))
    return "\n".join(results)

# Step 5: Choose best answer
choose_answer_prompt = PromptTemplate(
    input_variables=["asp_results", "original_question"],
    template="Given these ASP results:\n{asp_results}\n\nAnd the original question:\n{original_question}\n\nChoose the best answer:"
)

# Define the pipeline using LCEL with memory
memory = Memory()

pipeline = (
    RunnablePassthrough.assign(nl_input="A person wants to know if they can go to the park.")  # Replace with dynamic input
    | LangChainPredict(prompt=nl_to_facts_prompt, llm=llm)  # Convert to atomic facts
    | memory.store("atomic_facts")  # Store atomic facts in memory
    | RunnablePassthrough.assign(atomic_facts=lambda x: x)  # Pass atomic facts to the next step
    | LangChainPredict(prompt=asp_rules_prompt, llm=llm)  # Generate ASP program
    | LangChainPredict(prompt=debug_asp_prompt, llm=llm)  # Debug ASP program
    | RunnablePassthrough.assign(asp_program=lambda x: x)  # Pass debugged ASP program to the next step
    | Runnable(lambda x: run_asp_program(x))  # Run ASP program
    | memory.store("asp_results")  # Store ASP results in memory
    | LangChainPredict(prompt=choose_answer_prompt, llm=llm)  # Choose the best answer
)

# Execute the pipeline
if __name__ == "__main__":
    original_question = "Can I go to the park if it is raining?"
    result = pipeline.invoke({"nl_input": "A person wants to know if they can go to the park.", "original_question": original_question})
    print("Best Answer:", result)